In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import os, json, gc, re, random

import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
movie_df = pd.read_csv("kinopoisk-top250g.csv", sep=',')

movies = movie_df

In [ ]:
#оставим только жанры, которые встречаются больше 5 раз

In [3]:
shortlisted_genres = movies["Genre"].value_counts().reset_index(name="count").query("count > 5")["index"].tolist()
movies = movies[movies["Genre"].isin(shortlisted_genres)].reset_index(drop=True)

In [ ]:
#нормализация

In [4]:
movies['GenreCorrected'] =movies['Genre'] 
movies['GenreCorrected']=movies['GenreCorrected'].str.strip()
movies['GenreCorrected']=movies['GenreCorrected'].str.replace('драма', 'Драма')
movies['GenreCorrected']=movies['GenreCorrected'].str.replace('Драма ', 'Драма')
movies['GenreCorrected']=movies['GenreCorrected'].str.replace('Детективный', 'Детектив')

In [5]:
movies['GenreCorrected'].unique()

array(['Драма', 'Военный', 'Комедия', 'Боевик', 'Детский', 'Криминал',
       'фантастика', 'Фэнтези', 'Приключения', 'Триллер', 'Романтика',
       'Мюзикл'], dtype=object)

In [6]:
clas=[1] * len(movies['GenreCorrected'])
for i in range(len(movies['GenreCorrected'])):
  if movies['GenreCorrected'][i] == 'Драма':
    clas[i] = 0
  if movies['GenreCorrected'][i] == 'Военный':
    clas[i] = 1
  if movies['GenreCorrected'][i] == 'Комедия':
    clas[i] = 2
  if movies['GenreCorrected'][i] == 'Боевик':
    clas[i] = 3
  if movies['GenreCorrected'][i] == 'Детский':
    clas[i] = 4
  if movies['GenreCorrected'][i] == 'Криминал':
    clas[i] = 5
  if movies['GenreCorrected'][i] == 'фантастика':
    clas[i] = 6
  if movies['GenreCorrected'][i] == 'Фэнтези':
    clas[i] = 7
  if movies['GenreCorrected'][i] == 'Приключения':
    clas[i] = 8
  if movies['GenreCorrected'][i] == 'Триллер':
    clas[i] = 9
  if movies['GenreCorrected'][i] == 'Романтика':
    clas[i] = 10
  if movies['GenreCorrected'][i] =='Мюзикл':
    clas[i] = 11

In [7]:
movies['Genre_int'] = clas

In [8]:
movies = movies.drop(['rating', 'movie','year', 'country', 'rating_ball', 'director', 'screenwriter', 'actors', 'url_logo', 'Genre'], axis = 1)
movies.head()

,Plot,GenreCorrected,Genre_int
0,Бухгалтер Энди Дюфрейн обвинён в убийстве собс...,Драма,0
1,Пол Эджкомб — начальник блока смертников в тюр...,Драма,0
2,От лица главного героя Форреста Гампа; слабоум...,Драма,0
3,Фильм рассказывает реальную историю загадочног...,Военный,1
4,Пострадав в результате несчастного случая; бог...,Комедия,2


In [9]:
#делим выборку

from sklearn.model_selection import train_test_split

Y = movies['GenreCorrected']
X_train, X_test, y_train, y_test = train_test_split(movies['Plot'],Y, test_size = 0.1, random_state = 42)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(201,) (201,)
(23,) (23,)


In [10]:
movies.shape

(224, 3)

In [11]:
test_dataset = pd.DataFrame([y_test, X_test]).transpose()#.reset_index(drop=True)
test_dataset.head()

,GenreCorrected,Plot
9,Комедия,Инженер-изобретатель Тимофеев сконструировал м...
84,Романтика,Могучие ветры Гражданской войны в один миг уно...
117,Драма,Инженер Бен отправляется в необычное путешеств...
144,Романтика,Застенчивый и меланхоличный Джоэл живёт ничем ...
221,Военный,Флера — шестнадцатилетний мальчишка; откопавши...


In [12]:
movies = movies.drop(test_dataset.index)
movies.shape

(201, 3)

In [13]:
test_dataset = test_dataset.reset_index(drop = True)
test_dataset.head()

,GenreCorrected,Plot
0,Комедия,Инженер-изобретатель Тимофеев сконструировал м...
1,Романтика,Могучие ветры Гражданской войны в один миг уно...
2,Драма,Инженер Бен отправляется в необычное путешеств...
3,Романтика,Застенчивый и меланхоличный Джоэл живёт ничем ...
4,Военный,Флера — шестнадцатилетний мальчишка; откопавши...


In [ ]:
#используем BertTokenizer

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
#80 символов максимальное количество в plot

In [15]:
token = tokenizer.encode_plus(
    movies['Plot'].iloc[0], 
    max_length=80, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True, #[CLS],[PAD],[SEP]
    return_tensors='tf'
)
     

In [16]:
#создаем inputs ids и attention masks
X_input_ids = np.zeros((len(movies), 80))
X_attn_masks = np.zeros((len(movies), 80))

In [17]:
#функция генерации тренировочных данных
def generate_training_data(movies, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(movies['Plot'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=80, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [18]:
X_input_ids, X_attn_masks = generate_training_data(movies, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [19]:
#use labels as a one hot encoded target vector
labels = np.zeros((len(movies), 12))
labels.shape

(201, 12)

In [20]:
labels[np.arange(len(movies)), movies['Genre_int'].values] = 1 # one-hot encoded target tensor

In [ ]:
#преобразуем входные данные для чтения моделью BERT

In [21]:
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))


In [22]:
def DatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels    

In [23]:
dataset = dataset.map(DatasetMapFunction) 

In [ ]:
#перемешиваем датасет и устанавливаем размер батча

In [24]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) 

In [25]:
#80% тренировочная выборка, 20% тестовая
p = 0.8
train_size = int((len(movies)//16)*p)

In [26]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [ ]:
# bert base model with pretrained weights

In [27]:
model = TFBertModel.from_pretrained('bert-base-cased') 


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [28]:
input_ids = tf.keras.layers.Input(shape=(80,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(80,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(80, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(12, activation='softmax', name='output_layer')(intermediate_layer) 

movie_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
movie_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 80)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 80)]         0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 80,                                            

In [ ]:
#оптимизатор, функция потерь, accuracy

In [29]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')    

In [30]:
movie_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [31]:
hist = movie_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=25
)

Epoch 1/25
9/9 [==============================] - 12s 572ms/step - loss: 2.4272 - accuracy: 0.2361 - val_loss: 2.3931 - val_accuracy: 0.2292
Epoch 2/25
9/9 [==============================] - 4s 413ms/step - loss: 2.3332 - accuracy: 0.2708 - val_loss: 2.4087 - val_accuracy: 0.2292
Epoch 3/25
9/9 [==============================] - 4s 412ms/step - loss: 2.3530 - accuracy: 0.2639 - val_loss: 2.3340 - val_accuracy: 0.2292
Epoch 4/25
9/9 [==============================] - 4s 413ms/step - loss: 2.2272 - accuracy: 0.2986 - val_loss: 2.1828 - val_accuracy: 0.3333
Epoch 5/25
9/9 [==============================] - 4s 413ms/step - loss: 2.2582 - accuracy: 0.2778 - val_loss: 2.1412 - val_accuracy: 0.3542
Epoch 6/25
9/9 [==============================] - 4s 413ms/step - loss: 2.2311 - accuracy: 0.2917 - val_loss: 2.2930 - val_accuracy: 0.2500
Epoch 7/25
9/9 [==============================] - 4s 413ms/step - loss: 2.2321 - accuracy: 0.2778 - val_loss: 2.1835 - val_accuracy: 0.3333
Epoch 8/25
9/9 [===

In [32]:
movie_model.save('movie_model')

INFO:tensorflow:Assets written to: movie_model\assets


INFO:tensorflow:Assets written to: movie_model\assets


In [33]:
movie_model = tf.keras.models.load_model('movie_model')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=80, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['Драма', 'Военный', 'Комедия', 'Боевик', 'Детский', 'Криминал',
       'фантастика', 'Фэнтези', 'Приключения', 'Триллер', 'Романтика','Мюзикл']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

In [34]:
test_dataset['Predicted Genre'] = 'None'
for i in range(test_dataset.shape[0]):
    processed_data = prepare_data(test_dataset['Plot'].iloc[i], tokenizer)
    test_dataset['Predicted Genre'].iloc[i] = make_prediction(movie_model, processed_data=processed_data)

1/1 [==============================] - 0s 49ms/step


In [35]:
test_dataset

,GenreCorrected,Plot,Predicted Genre
0,Комедия,Инженер-изобретатель Тимофеев сконструировал м...,Драма
1,Романтика,Могучие ветры Гражданской войны в один миг уно...,Военный
2,Драма,Инженер Бен отправляется в необычное путешеств...,Военный
3,Романтика,Застенчивый и меланхоличный Джоэл живёт ничем ...,Криминал
4,Военный,Флера — шестнадцатилетний мальчишка; откопавши...,Драма
5,Военный,Последние дни Второй мировой войны; американск...,Драма
6,Военный,Рассказ о людях; в чьи судьбы безжалостно втор...,Криминал
7,фантастика,Каждые пять тысяч лет открываются двери между ...,Приключения
8,Комедия,В легендах и мифах есть персонажи; главная зад...,Боевик
9,Фэнтези,Жизнь десятилетнего Гарри Поттера нельзя назва...,Криминал
